In [35]:
from sympy import symbols, Eq, solve, Matrix
import sympy as sp
import numpy as np

## Iterasi 1 titik

In [82]:
'''
1. Tentukan fungsi utk func_x, func_y, .. dari fungsi utama 
'''
x = symbols('x')
y = symbols('y')
n = 4  # Adjust the n

xys = [
    [1, 1]
]

func_x = ((19 + y**2 - 4*y) / (2)) ** (1/3)
func_y = ((26 - x**2 - 2*x) / (2)) ** 0.5

for i in range(n):
    xi, yi = xys[-1]
    x_temp = round(func_x.subs({x: xi, y: yi}).evalf(), 5)
    y_temp = round(func_y.subs({x: xi, y: yi}).evalf(), 5)
    xys.append([x_temp, y_temp])
    
for iter, res in enumerate(xys):
    print(iter, res)

0 [1, 1]
1 [2.00000000000000, 3.39116]
2 [2.03824, 3.00000000000000]
3 [2.00000000000000, 2.98070]
4 [1.99841, 3.00000000000000]


## Newton

In [83]:
def find_first_jacobian(vars, equations, initial_guess):
    # variables = list(set().union(*[eq.free_symbols for eq in equations]))
    # jacobian = Matrix([[sp.diff(eq, var) for var in variables] for eq in equations])
    jacobian =[]
    
    for equation in equations:
        curr_calc = []
        for i, var in enumerate(vars):
            temp = sp.diff(equation, var)
            curr_calc.append(temp)
        jacobian.append(curr_calc)
        
    return jacobian

def find_jacobian_value(vars, xys, jacobian):
    js = []
    
    for iter, jacob in enumerate(jacobian):
        temp = []
        for j in jacob:
            dics = {}
            for sym, var in zip(vars, xys):
                dics[sym] = var
            temp.append(j.subs(dics).evalf())
        js.append(temp)
    return js

In [84]:
x = symbols('x')
y = symbols('y')
pembulatan = 5

func_x = 2 * x * x * x - y ** 2 + 4 * y - 19
func_y = x ** 2 + 2 * x + 2 * y ** 2 - 26

xys = [
    [1, 1]
]

# ===== INITIAL JACOBIAN =====
print("======================== JACOBIAN ==========================")
jacobian = find_first_jacobian([x, y], [func_x, func_y], xys)

first_approach = find_jacobian_value([x, y], xys[0], jacobian)

for values in first_approach:
    for pos, value in enumerate(values):
        if pos == 0:
            print("|", end="")
        if (pos < len(values) - 1):
            print(f"{round(float(value), 5):<12}", end="")
        else:
            print(f"{round(float(value), 5)}", end="")
        if pos == len(values) - 1:
            print("|", end="")
    print("")
print("")

# ===== F1 AND F2 =====
print("======================== F1 AND F2 ==========================")
f1 = func_x.subs({x: xys[0][0], y: xys[0][1]}).evalf()
f2 = func_y.subs({x: xys[0][0], y: xys[0][1]}).evalf()

print(f"f1(x0, y0): {round(float(f1), pembulatan)}\nf2(x0, y0): {round(float(f2), pembulatan)}\n")

# ===== FIRST APPROACH X AND Y =====
print("======================== VALUES OF X AND Y ==========================")
# iteration
iteration = 3

for i in range(iteration):
    X = np.array(xys[-1], dtype=float)  # Current guess for [x, y]
    F = np.array([f1, f2], dtype=float)  # Function values at current guess

    # Get the Jacobian matrix at the current guess
    jacobian_values = np.array(find_jacobian_value([x, y], xys[-1], jacobian), dtype=float)

    # Solve for the correction delta (J * delta = F, so we need to invert J)
    delta = np.linalg.solve(jacobian_values, F)  # Solving J * delta = F

    # Update X (x, y)
    X_new = X - delta  # Update using the correction delta
    print(f"Iteration {i + 1}: {X_new}")

    # Append the new [x, y] to the list for the next iteration
    xys.append(list(X_new))

    # Recalculate f1 and f2 for the new x, y for next iteration
    f1 = func_x.subs({x: X_new[0], y: X_new[1]}).evalf()
    f2 = func_y.subs({x: X_new[0], y: X_new[1]}).evalf()
    


======================== JACOBIAN ==========================
|6.0         2.0|
|4.0         4.0|

======================== F1 AND F2 ==========================
f1(x0, y0): -14.0
f2(x0, y0): -21.0

======================== VALUES OF X AND Y ==========================
Iteration 1: [1.875 5.375]
Iteration 2: [1.91640484 3.54779289]
Iteration 3: [1.99630525 3.0443017 ]
